In [13]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import linear_model
import itertools
from tqdm import tqdm
from sklearn.feature_selection import SelectKBest, f_regression, RFECV
from sklearn.model_selection import cross_val_score, KFold

In [2]:
df = pd.read_csv('logP.csv')
y = df.logP
X = df.iloc[:, 3:]
feats = X.columns

In [3]:
kf = KFold(n_splits = 5, shuffle = True, random_state = 42)

## KBest

In [26]:
selected_feature_names

Index(['MolWt', 'HeavyAtomMolWt', 'ExactMolWt', 'NumValenceElectrons',
       'BCUT2D_MWLOW', 'BCUT2D_CHGHI', 'BCUT2D_CHGLO', 'BCUT2D_LOGPHI',
       'AvgIpc', 'BalabanJ', 'BertzCT', 'Chi0', 'Chi0n', 'Chi0v', 'Chi1',
       'Chi1n', 'Chi1v', 'Chi2n', 'Chi2v', 'Chi3n', 'Chi3v', 'Chi4n', 'Chi4v',
       'HallKierAlpha', 'Ipc', 'Kappa1', 'Kappa2', 'Kappa3', 'LabuteASA',
       'PEOE_VSA6', 'PEOE_VSA7', 'SMR_VSA10', 'SMR_VSA4', 'SMR_VSA5',
       'SMR_VSA7', 'SlogP_VSA5', 'SlogP_VSA6', 'EState_VSA4', 'EState_VSA7',
       'EState_VSA8', 'VSA_EState6', 'HeavyAtomCount',
       'NumAliphaticCarbocycles', 'NumAliphaticRings',
       'NumAromaticCarbocycles', 'RingCount', 'MolMR', 'fr_ArN',
       'fr_aryl_methyl', 'fr_benzene'],
      dtype='object')

In [4]:
selector = SelectKBest(f_regression, k = 50)
study = selector.fit_transform(X, y) 
selected_feature_indices = selector.get_support()
selected_feature_names = X.columns[selected_feature_indices]
X_new = pd.DataFrame(study, columns = selected_feature_names)
feats2 = X_new.columns
X_new

,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,BCUT2D_MWLOW,BCUT2D_CHGHI,BCUT2D_CHGLO,BCUT2D_LOGPHI,AvgIpc,BalabanJ,...,VSA_EState6,HeavyAtomCount,NumAliphaticCarbocycles,NumAliphaticRings,NumAromaticCarbocycles,RingCount,MolMR,fr_ArN,fr_aryl_methyl,fr_benzene
0,281.355,262.203,281.141579,108.0,9.826610,2.283329,-2.277418,2.409873,2.479817,2.167149,...,12.173224,21.0,0.0,1.0,2.0,3.0,83.9997,0.0,2.0,2.0
1,262.183,253.111,262.045293,98.0,9.976269,2.595214,-2.338667,2.417638,2.260441,2.466824,...,3.854776,18.0,0.0,1.0,1.0,2.0,53.5743,0.0,0.0,1.0
2,266.388,244.212,266.178299,104.0,9.988563,2.037722,-2.148481,2.248038,2.463877,2.219002,...,16.969155,20.0,0.0,0.0,2.0,2.0,89.6740,0.0,0.0,2.0
3,318.372,300.228,318.125594,120.0,9.864935,2.289913,-2.232562,2.289585,3.072296,2.024811,...,21.820029,24.0,0.0,0.0,2.0,3.0,92.0636,0.0,1.0,2.0
4,214.312,196.168,214.146999,84.0,9.917986,2.159146,-2.141439,2.206135,2.671659,2.150835,...,12.581179,16.0,0.0,0.0,1.0,2.0,66.5357,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,381.501,358.317,381.151098,140.0,10.109655,2.366746,-2.103953,2.324511,2.859261,1.720645,...,13.598590,27.0,0.0,1.0,2.0,3.0,111.5857,0.0,3.0,2.0
240,321.376,302.224,321.136493,122.0,10.109605,2.076714,-2.115232,2.259625,2.610351,1.626537,...,21.145723,24.0,0.0,0.0,3.0,3.0,95.6837,0.0,0.0,3.0
241,274.266,262.170,274.080536,102.0,9.866062,2.290398,-2.231605,2.289251,2.459367,2.551777,...,10.369830,20.0,0.0,0.0,2.0,2.0,70.6606,0.0,0.0,2.0
242,321.312,308.208,321.088080,120.0,10.152994,2.044913,-2.091641,2.192387,2.554374,1.929950,...,13.230576,24.0,0.0,0.0,2.0,3.0,89.9297,0.0,0.0,2.0


## L1

In [5]:
def feature_selection_L1(X, y, features: list, n_features = 2):

    r2_values = []

    for n_feat in range(1, n_features+1):

        combs = itertools.combinations(features, n_feat)

        for feature_subset in combs:

            model = linear_model.LassoCV(cv = kf)

            X_train = X[[*feature_subset]]

            model.fit(X_train, y)
            r2_value = model.score(X_train, y)

            r2_values.append((n_feat, r2_value, ' '.join(feature_subset)))

    r2_df = pd.DataFrame(r2_values, columns=['n', 'r2', 'cols'])

    return r2_df.groupby('n').max()


In [8]:
reg_l1 = feature_selection_L1(X_new, y, feats2, 2)
reg_l1

/home/stas/.local/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.040e-02, tolerance: 2.275e-02
  model = cd_fast.enet_coordinate_descent(
/home/stas/.local/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.301e-02, tolerance: 2.275e-02
  model = cd_fast.enet_coordinate_descent(


,r2,cols
n,,
1,0.328713,fr_benzene
2,0.398445,fr_aryl_methyl fr_benzene


## L2

In [9]:
def feature_selection_L2(X, y, features: list, n_features = 2):

    r2_values = []

    for n_feat in tqdm(range(1, n_features+1)):

        combs = itertools.combinations(features, n_feat)

        for feature_subset in combs:

            model = linear_model.RidgeCV(cv = kf)

            X_train = X[[*feature_subset]]

            model.fit(X_train, y)
            r2_value = model.score(X_train, y)

            r2_values.append((n_feat, r2_value, ' '.join(feature_subset)))

    r2_df = pd.DataFrame(r2_values, columns=['n', 'r2', 'cols'])

    return r2_df.groupby('n').max()


In [10]:
reg_L2 = feature_selection_L2(X_new, y, feats2, 2)
reg_L2

  0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:20<00:00, 10.31s/it]


,r2,cols
n,,
1,0.328713,fr_benzene
2,0.398446,fr_aryl_methyl fr_benzene


## NO

In [11]:
def feature_selection_L0(X, y, features: list, n_features = 2):

    r2_values = []

    for n_feat in range(1, n_features+1):

        combs = itertools.combinations(features, n_feat)

        for feature_subset in combs:

            model = linear_model.LinearRegression()

            X_train = X[[*feature_subset]]

            model.fit(X_train, y)
            r2_value = model.score(X_train, y)

            r2_values.append((n_feat, r2_value, ' '.join(feature_subset)))

    r2_df = pd.DataFrame(r2_values, columns=['n', 'r2', 'cols'])

    return r2_df.groupby('n').max()


In [12]:
reg_L0 = feature_selection_L0(X_new, y, feats2, 2)
reg_L0

,r2,cols
n,,
1,0.328713,fr_benzene
2,0.398446,fr_aryl_methyl fr_benzene


## RFECV

In [14]:
model = linear_model.LinearRegression()
rfecv = RFECV(estimator = model, min_features_to_select = 25, cv = kf)

rfecv.fit(X, y)

selected_features = X.columns[rfecv.support_]
X_new2 = X[selected_features]
feats22 = X_new2.columns
X_new2

,QED_score,qed,MolWt,ExactMolWt,MinPartialCharge,MaxAbsPartialCharge,MinAbsPartialCharge,FpDensityMorgan1,FpDensityMorgan2,BCUT2D_LOGPLOW,...,EState_VSA5,EState_VSA6,EState_VSA7,fr_N_O,fr_dihydropyridine,fr_imidazole,fr_ketone,fr_ketone_Topliss,fr_oxime,fr_tetrazole
0,0.909,0.909253,281.355,281.141579,-0.496454,0.496454,0.224838,1.095238,1.809524,-2.237256,...,16.690354,7.109798,24.265468,0,0,0,0,0,0,0
1,0.839,0.839542,262.183,262.045293,-0.496624,0.496624,0.455288,1.444444,2.111111,-2.533527,...,19.242532,0.000000,0.000000,0,0,0,1,1,0,0
2,0.828,0.828333,266.388,266.178299,-0.377661,0.377661,0.036092,0.650000,0.950000,-2.080683,...,11.374773,0.000000,28.190688,0,0,0,0,0,0,0
3,0.726,0.726080,318.372,318.125594,-0.462534,0.462534,0.176481,0.875000,1.416667,-2.249719,...,0.000000,19.056471,60.663671,0,0,0,0,0,0,0
4,0.816,0.816043,214.312,214.146999,-0.281751,0.281751,0.066847,1.125000,1.812500,-2.107302,...,11.257379,0.000000,6.066367,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,0.863,0.863227,381.501,381.151098,-0.326039,0.326039,0.241916,1.148148,1.814815,-2.296841,...,11.761885,11.947582,63.235781,0,0,0,0,0,0,0
240,0.737,0.737878,321.376,321.136493,-0.489991,0.489991,0.261809,0.875000,1.541667,-2.198811,...,0.000000,6.066367,67.587408,0,0,0,0,0,0,0
241,0.824,0.824192,274.266,274.080536,-0.383653,0.383653,0.176462,0.850000,1.250000,-2.250916,...,48.530937,0.000000,0.000000,0,0,0,0,0,0,0
242,0.750,0.750035,321.312,321.088080,-0.578172,0.578172,0.220747,1.166667,1.875000,-2.120952,...,13.138338,48.530937,0.000000,0,0,0,0,0,0,0


## L1

In [15]:
def feature_selection_L1_RFECV(X, y, features: list, n_features = 2):

    r2_values = []

    for n_feat in tqdm(range(1, n_features+1)):

        combs = itertools.combinations(features, n_feat)

        for feature_subset in combs:

            model = linear_model.LassoCV(cv = kf)

            X_train = X[[*feature_subset]]

            model.fit(X_train, y)
            r2_value = model.score(X_train, y)

            r2_values.append((n_feat, r2_value, ' '.join(feature_subset)))

    r2_df = pd.DataFrame(r2_values, columns=['n', 'r2', 'cols'])

    return r2_df.groupby('n').max()


In [16]:
res_L1_RFECV = feature_selection_L1_RFECV(X_new2, y, feats22, 2)
res_L1_RFECV

 50%|█████     | 1/2 [00:00<00:00,  2.04it/s]/home/stas/.local/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:614: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.5776274835878041, tolerance: 0.019063452615384616
  model = cd_fast.enet_coordinate_descent_gram(
/home/stas/.local/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:614: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.338952370577829, tolerance: 0.019063452615384616
  model = cd_fast.enet_coordinate_descent_gram(
/home/stas/.local/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:614: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 7.332202508092649, tolerance: 0.019063452615384616
  model = cd_fast.enet_coordinate_descent_gram(
/home/stas/.local/lib/pytho

,r2,cols
n,,
1,0.302002,qed
2,0.338641,qed fr_tetrazole


## L2

In [17]:
def feature_selection_L2_RFECV(X, y, features: list, n_features = 2):

    r2_values = []

    for n_feat in tqdm(range(1, n_features+1)):

        combs = itertools.combinations(features, n_feat)

        for feature_subset in combs:

            model = linear_model.RidgeCV(cv = kf)

            X_train = X[[*feature_subset]]

            model.fit(X_train, y)
            r2_value = model.score(X_train, y)

            r2_values.append((n_feat, r2_value, ' '.join(feature_subset)))

    r2_df = pd.DataFrame(r2_values, columns=['n', 'r2', 'cols'])

    return r2_df.groupby('n').max()


In [18]:
res_L2_RFECV = feature_selection_L2_RFECV(X_new2, y, feats22, 2)
res_L2_RFECV

100%|██████████| 2/2 [00:07<00:00,  3.63s/it]


,r2,cols
n,,
1,0.302003,qed
2,0.338643,qed fr_tetrazole


## NO

In [21]:
def feature_selection_NO_RFECV(X, y, features: list, n_features = 2):

    r2_values = []

    for n_feat in tqdm(range(1, n_features+1)):

        combs = itertools.combinations(features, n_feat)

        for feature_subset in combs:

            model = linear_model.LinearRegression()

            X_train = X[[*feature_subset]]

            model.fit(X_train, y)
            r2_value = model.score(X_train, y)

            r2_values.append((n_feat, r2_value, ' '.join(feature_subset)))

    r2_df = pd.DataFrame(r2_values, columns=['n', 'r2', 'cols'])

    return r2_df.groupby('n').max()


In [22]:
res_NO_RFECV = feature_selection_NO_RFECV(X_new2, y, feats22, 2)
res_NO_RFECV

  0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:00<00:00,  4.46it/s]


,r2,cols
n,,
1,0.302003,qed
2,0.338643,qed fr_tetrazole
